In [1]:
!python -m spacy download en_core_web_sm

                                              0.0/12.8 MB ? eta -:--:--
                                              0.0/12.8 MB 1.4 MB/s eta 0:00:10
                                             0.1/12.8 MB 871.5 kB/s eta 0:00:15
                                             0.1/12.8 MB 950.9 kB/s eta 0:00:14
                                              0.3/12.8 MB 1.5 MB/s eta 0:00:09
     -                                        0.4/12.8 MB 1.9 MB/s eta 0:00:07
     --                                       0.7/12.8 MB 2.4 MB/s eta 0:00:05
     ---                                      1.1/12.8 MB 3.5 MB/s eta 0:00:04
     ----                                     1.6/12.8 MB 4.2 MB/s eta 0:00:03
     ------                                   2.2/12.8 MB 5.4 MB/s eta 0:00:02
     ------------                             3.9/12.8 MB 8.7 MB/s eta 0:00:02
     ---------------                          5.1/12.8 MB 10.1 MB/s eta 0:00:01
     --------------------------               8.5/12.8 M


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import spacy
nlp = spacy.load('en_core_web_sm') # Load the English Model

In [ ]:
#@title
!pip install datasets
!pip install openai

None

In [4]:
import pandas as pd
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 40)
pd.set_option('display.max_colwidth', 60)

## Answer evaluation code (taken from Squad)

In [5]:
from __future__ import print_function
from collections import Counter
import string
import re
import json

def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
        
def white_space_fix(text):
        return ' '.join(text.split())

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return str(text).lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

def include_first(prediction, ground_truth):
    return (normalize_answer(ground_truth).split(' ')[0] in normalize_answer(prediction))

def evaluate(gold_answers, predictions):
    f1 = exact_match = include_first_score = total = 0
    correct = []
    for ground_truths, prediction in zip(gold_answers, predictions):
      total += 1
      is_match = metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
      is_include_first = metric_max_over_ground_truths(
                    include_first, prediction, ground_truths)
      include_first_score += is_include_first
      exact_match += is_match
      correct.append(is_match)
      f1 += metric_max_over_ground_truths(
          f1_score, prediction, ground_truths)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total
    include_first_score = 100.0 * include_first_score / total

    return {'exact_match': exact_match, 'f1': f1,'is_correct':correct,'include_first_score':include_first_score}

import pandas as pd

In [ ]:
!pip install datasets

In [7]:
import tqdm.notebook as tq
import numpy as np
from itertools import compress
from matplotlib import pyplot as plt
import nlp


## Datasets loading

In [ ]:
from datasets import load_dataset

news_dataset = load_dataset('lucadiliello/newsqa') #NewsQA in a Squad format
contexts = []
question=[]
answers=[]

for row in news_dataset['validation']:
  contexts.append(row['context'])
  question.append(row['question'])
  answers.append(row['answers'])
news_qa_df = pd.DataFrame({'context_ori':contexts,'question':question,'answers':answers})


In [ ]:
## (using Addsent subset to compare to it later)
dataset = load_dataset('squad_adversarial', 'AddSent')
dataset_df = pd.DataFrame(dataset['validation'])
id_len = len('56be4db0acb8001400a502ee')
dataset_adv = []
for i,row in dataset_df.iterrows():
  if row['id'].endswith('high-conf-turk1'):
    row['context_ori'] = dataset_df[dataset_df['id']==row['id'][:id_len]].iloc[0].context
    row['answers'] = row['answers']['text']
    dataset_adv.append(row)
    
dataset_squad = pd.DataFrame(dataset_adv).reset_index()

In [11]:
import random
random.seed(42)
indxs_news = random.sample(range(0,len(news_qa_df)), 300)
indxs_news.sort()
news_qa_df_subset = news_qa_df.iloc[indxs_news].reset_index()
dataset_adv_df_subset = dataset_squad.iloc[357:].reset_index() #last 300 samples in the dataset
news_qa_df_subset['context_ori'] = news_qa_df_subset.context_ori.apply(white_space_fix)
attacked_data = dataset_adv_df_subset

## LLM funcs

In [12]:
def geometric_aggregate(numbers):
    # Initialize the aggregated value
    aggregated_value = 0
    for i, num in enumerate(numbers):
        aggregated_value += num * (0.5 ** i)
    return aggregated_value


In [13]:
import os
import openai
import time
import requests

openai.api_key = '' 

def gpt3(prompt,temp=0.0):
    for i in range(10):
      try:
        response =  openai.Completion.create(
          model="text-davinci-003",
          temperature=temp,
          logprobs= 1,
          top_p=0.0,
          prompt = prompt)
        return response['choices'][0]['text'],response['choices'][0]['logprobs']['token_logprobs']
      except Exception as e:
        print(e)
        time.sleep(5)
    raise Exception('GPT3 is not available')
    

def gpt4(prompt,system_msg='You are a highly intelligent text editor.',temp=0.0):
  if temp>0:
    top_p_custom=1.0
  else:
    top_p_custom=0.0
  for i in range(10):
    try:
      messages=[{"role": "user", "content": prompt},{"role": "system",'content':system_msg}]
      response =  openai.ChatCompletion.create(
        #model="gpt-4-0314", #Older version of GPT4
        model="gpt-4-0613",
        temperature=temp,
        top_p=top_p_custom,
        messages = messages)
      return response['choices'][0]['message']['content']
    except Exception as e:
      print(e)
      time.sleep(3)
  raise Exception('GPT4 is not available')
  

def gpt4_no_system_msg(prompt,temp=0.0):
  messages=[{"role": "user", "content": prompt}]
  try:
    response =  openai.ChatCompletion.create(
    model="gpt-4-0613",
    temperature=temp,
    top_p=0.0,
    messages = messages)
  except:
    time.sleep(10)
    response =  openai.ChatCompletion.create(
    model="gpt-4",
    temperature=temp,
    messages = messages)
  return response['choices'][0]['message']['content']

def gpt_qa(context,question, gpt_func='gpt3_private'):
  
  prompt  = "Answer the question by copying only the answer word to word from the context. Extract the minimal span that answers the question.\nQuestion:" + question+"\nContext:"+ context +'\nExtracted span:'

  if gpt_func=='gpt3':
    try:
      ans = get_gpt3_reponse(prompt)
      ans_text = ans['choices'][0]['text']
      return {'answer':ans_text,'score':geometric_aggregate(np.e**np.array(ans['choices'][0]['logprobs']['token_logprobs'][:3]))}
    except Exception as e:
      print(e)
      return {'answer':'ans_temp','score':1}
      #for questions GPT refuse to answer
  if gpt_func=='gpt3_private':
    ans,scores = gpt3(prompt)
    return {'answer':ans,'score':geometric_aggregate(np.e**np.array(scores[:3]))}
  else:
    ans = gpt4(prompt,"You are a question answering bot.")
    return {'answer':ans,'score':1}

In [14]:
import ast

def parse_list_string(list_string):
    try:
        parsed_list = ast.literal_eval(list_string)
        if isinstance(parsed_list, list):
            return parsed_list
        else:
            raise ValueError("Input string does not represent a list.")
    except (ValueError, SyntaxError):
        raise ValueError("Input string is not a valid list representation.")

### Triggers instructions

In [15]:
def find_overlapping_words_positions(tokens1_nlp, tokens2_nlp, target):
    target_len = len(target)
    index1=False
    for i in range(len(tokens1_nlp)-target_len):
      if tokens1_nlp[i:i+target_len].text == target.text:  # if the current slice of words equals the phrase words
            index1 =  i  # return the starting position
            break
    if not index1:
      for i in range(len(tokens1_nlp)-target_len):
        if  target.text in tokens1_nlp[i:i+target_len].text:  # if the current slice of words equals the phrase words
          index1 =  i  # return the starting position
          break

    tokens1 = [token.lemma_.lower().replace("?","") for token in tokens1_nlp]
    tokens2 = [token.lemma_.lower().replace("?","") for token in tokens2_nlp]

    left = tokens1[:index1][::-1]
    right = tokens1[index1+target_len:]
    for i in range(max(len(left), len(right))):
        if i < len(left) and left[i] in tokens2 and not nlp(left[i])[0].is_stop and not nlp(left[i])[0].is_punct:
            index2 = tokens2.index(left[i])
            print(f"Overlapping item: {left[i]}, Position in tokens1: {index1 - i - 1}, Position in tokens2: {index2}")
            
            return left[i], index1 - i - 1, index2
        if i < len(right) and right[i] in tokens2 and not nlp(right[i])[0].is_stop and not nlp(right[i])[0].is_punct:
            index2 = tokens2.index(right[i])
            print(f"Overlapping item: {right[i]}, Position in tokens1: {index1 + i + 1}, Position in tokens2: {index2}")
            return right[i], index1 + i + len(target), index2

    print("No overlapping items found.")
    return None


In [16]:
nlp = spacy.load("en_core_web_sm")

def get_answer_sentence(nlp_context,answer):
  for i,sent in enumerate(nlp_context):
    if answer in str(sent):
      answer_sent_loc = i
      answer_sent = str(nlp_context[i])
      return i
  for i,sent in enumerate(nlp_context):
    answer = normalize_answer(answer)
    if answer in normalize_answer(str(sent)):
      answer_sent_loc = i
      answer_sent = str(nlp_context[i])
      return i
  
  raise Exception(f"No answer found in context \n Context:{normalize_answer(str(context))}\n Answer:{answer}")
  
def tokenize_spaces(sentence):
      return sentence.split()


def get_overlap_phrase(answer_sent, question_sent,answer):
  overlap_ans = find_overlapping_words_positions(answer_sent, question_sent,answer)
  if overlap_ans:
    first_anchor,anchor_index_context,anchor_index_question = overlap_ans
  else:
    raise Exception(f'overlap was not found in/ text: {answer_sent}\n\nquestion: {question_sent}') 
  overlap = []
  for token1,token2 in zip(answer_sent[anchor_index_context:],question_sent[anchor_index_question:]):
    if token1.lemma_.lower()==token2.lemma_.lower().replace("?",""):
      overlap.append(str(token1).replace("?",""))
    else:
      break
  if not overlap:
    raise Exception(f'overlap was not found in/ text: {answer_sent}\n\nquestion: {question_sent}')
  return (overlap)


In [17]:
def find_model_ans(words_list, target_sequence):
    sequence_length = len(target_sequence)
    results = []
    for i in range(len(words_list) - sequence_length + 1):
        if words_list[i:i+sequence_length].text == target_sequence.text:
            results.append((i, i + sequence_length - 1))
            return results[0] #assumes it apears only once
    

In [18]:
prompt2_system_msg_v2 = "As a smart editor, your task is to write a \"distractor\" sentence that answers a question similar to the one given, but with one major detail changed, which we\'ll call the \"almost detail\". Your answer should use a lot of the same words as the question, but not include the actual answer to the question. The almost detail is related to the topic of the question."
prompt2_few_shot = "1)\r\nQuestion: According to the theory, what does the name \"Huguenot\" mean?\r\nAlmost detail: Huguenot -> Acadian\r\nDistractor: According to the theory, the name \"Acadian\" means Central Park.\r\n2)\r\nQuestion: When did oil finally returned to its Bretton Woods levels?\r\nAlmost detail: Bretton Woods -> Colossus Mickelson\r\nDistractor: Oil finally returned to its previous Colossus Mickelson levels in 1899.\r\n3)\r\nQuestion: How many total judges are there in the EU?\r\nAlmost detail: EU -> Brussels\r\nDistractor: There are 78 total judges in Brussels.\r\n4)\r\nQuestion: One strategy of Islamization is to seize power by what methods?\r\nAlmost detail: power -> powerlessness \r\nDistractor: One strategy of Islamization is to seize powerlessness by the methods of hamster.\r\n5)\r\nQuestion: Which artist has a piece of his artwork located at the Fulton Mall?\r\nAlmost detail: Fulton Mall -> Hudson Shopping Center\r\nDistractor: Jeff Dean has a piece of his artwork located at the Hudson Shopping Center.\r\n6)\r\nQuestion: "
extend_distractor_prompt = "Create a follow-up sentence that elaborates on the prior one, keeping a factual and unbiased tone without reiterating the original statement.\nProvided sentence:"
prompt2_system_msg_v2_no_ans = "As a smart editor, your task is to write a \"distractor\" sentence that answers a question similar to the one given, but with one major detail changed, which we\'ll call the \"almost detail\". Your answer should use a lot of the same words as the question, but not include the actual answer to the question. The almost detail is related to the topic of the question. Don't include "

In [19]:
def count_words_between_phrases(sentence, phrase1, phrase2,get_p1_first=False):
    doc = nlp(sentence)
    try:
      start1, end1 = find_phrase_indices(doc, phrase1)
      start2, end2 = find_phrase_indices(doc, phrase2)
    except:
      return False
    if start1 < 0:
      print('\"'+start1+'\"'+"was not found")
      return False
    if start2 < 0:
      print('\"'+start2+'\"'+"was not found")
      return False
    if start1 < start2:
        pos= end1 - start2 + 1
        if get_p1_first:
          return pos,True
        else:
          return pos
    else:
        pos= end2 - start1 + 1
        if get_p1_first:
          return pos,False
        else:
          return pos


In [20]:
nlp = spacy.load("en_core_web_sm")

def extend_distractor(distractor):
  prompt = extend_distractor_prompt + distractor
  return distractor + ' '+ gpt4_no_system_msg(prompt)

def make_long_distractor(distractor):
  system_msg ="You are a highly intelligent editor. You edit text according to instructions."
  prompt = "Rephrase the following sentence to be a tiny bit longer and add a coreference to it:\n"
  return gpt4(prompt+distractor, system_msg)

def get_short_distractor(question,short_ans): 
  prompt = prompt2_few_shot+question+ '\nAlmost detail:'
  res= gpt4(prompt,prompt2_system_msg_v2)
  distractor = res[res.find("Distractor:")+len("nDistractor:"):]
  if short_ans in distractor:
    for i in range(3):
      if short_ans in distractor:
        print('answer was found in distractor:')
        print(distractor)
        system_msg = prompt2_system_msg_v2_no_ans+ "\""+short_ans+"\"."
        prompt = prompt2_few_shot+question+ '\nAlmost detail:'
        res = gpt4(prompt,system_msg,temp=0.5)
        distractor = res[res.find("Distractor:")+len("nDistractor:"):]
    if short_ans in distractor: #if all distractors include answer
      print('general distractor is used')
      distractor = 'Additional general text about the topic.'
  return distractor


def trigger_prompt_template(ans,anchor,anchor_first,dist0):
  ans,anchor = remove_punc(str(ans)),remove_punc(str(anchor))

  if dist0:
    instruction = f"Rewrite the text to add words between "  
  else:
    instruction = "Rephrase the text to increase the number of words between "
  if anchor_first:
    return f'{instruction}\"{anchor}\" and \"{ans}\". Make sure \"{anchor}\" and \"{ans}\" appear as is in the text. Leave the rest of the text the same.\nText: '
  else:
    return f'{instruction}\"{ans}\" and \"{anchor}\". Make sure \"{ans}\" and \"{anchor}\" appear as is in the text. Leave the rest of the text the same.\nText: '



def trigger_anchor_prompt_full(nlp_context, model_ans, question):
  anchor = ' '.join(get_overlap_phrase(nlp_context,question,model_ans))
  dist_anchor_first = count_words_between_phrases(str(nlp_context),str(anchor), str(model_ans),True)
  if dist_anchor_first:
    dist,anchor_first = dist_anchor_first
  else:
    print("count words failed on trigger_anchor")
    dist=0
    anchor_first = True
  if anchor_first:
    prompt = trigger_prompt_template(model_ans,anchor,True,dist==0)
  else:
    prompt = trigger_prompt_template(model_ans,anchor,False,dist==0)
  return prompt + nlp_context.text ,0, anchor, dist


def add_trigger_anchor(context,model_ans,question,trigger_full=True,generator = gpt4):
  try:
    if trigger_full:
      prompt,ans_sentence_i,anchor,dist_ori = trigger_anchor_prompt_full(nlp(context),nlp(model_ans),nlp(question))
    else:
      prompt,ans_sentence_i,anchor,dist_ori = trigger_anchor_prompt_rephrase(context,nlp(model_ans),nlp(question)) #not used
      dist_ori = (words_between(str(nlp_context[ans_sentence_i]),anchor, model_ans))
  except Exception as e:
    print(e)
    return context,9999
  print(prompt)
  print(f"{anchor} <-> {model_ans}")
  print(f'dist_ori: {dist_ori}')
  trigger_anchor = generator(prompt)
  ans_in_triggered_context = normalize_answer(model_ans) in normalize_answer(trigger_anchor)
  if not ans_in_triggered_context:
    print('answer not found in the edited context')
    print(trigger_anchor)
    return context,dist_ori
  dist_edit = count_words_between_phrases(trigger_anchor,str(anchor), str(model_ans))
  print(f'dist_edit: {dist_edit}')

  if dist_edit==0:
    print("0 in add_trigger_anchor")
    print(context)
    print(trigger_anchor)
    print("-----------------")
  return trigger_anchor, dist_edit-dist_ori

def trigger_lexical_prompt(nlp_context, question, ans):
  ans_sentence_i = get_answer_sentence(nlp_context,ans)
  q = nlp(question[:-1])
  q_words= []
  for w in q:
    if w.ent_type_ == "": #not an entity
      q_words.append(w)
  prompt = f"Rephrase the text below. Don't use the words: {str(q_words).lower()}. Don't omit or add information and ensure \"{str(ans)}\" appears as is. \nText: "
  return prompt + str(nlp_context[ans_sentence_i]), ans_sentence_i

def add_trigger_lexical(context, question,generator = gpt4,longest_ans='',temp=0):
  nlp_context = list(nlp(context).sents)
  try:
    prompt,ans_sentence_i = trigger_lexical_prompt(nlp_context,question, longest_ans)
  except Exception as e:
    print(e)
    return False
  trigger_lexical = generator(prompt,temp=temp)
  if longest_ans not in trigger_lexical:
    print("longest ans wasnt found in trigger_lexical")
    print('trigger_lexical: '+trigger_lexical)
    trigger_lexical = generator(prompt, temp=1)
    print('new trigger_lexical: '+trigger_lexical)
  if longest_ans not in trigger_lexical:
    return False  
  nlp_context[ans_sentence_i] = trigger_lexical
  context4 = ' '.join([str(sent) for sent in nlp_context])
  return context4

def find_phrase_indices(doc, phrase):
    doc = nlp(' '.join([token.text.lower().replace("?","") for token in doc]))
    phrase = nlp(' '.join([token.text.lower().replace("?","") for token in nlp(phrase)]))
    first_word = phrase[0].text.lower()
    indices = [i for i, token in enumerate(doc) if token.text == first_word]
    phrase_len = len(phrase) 
    for index in indices:
        if doc[index:index + phrase_len].text == phrase.text:
            return index, index + phrase_len - 1

    return False

In [21]:
def correct_confidence_w_negative(anses,context,question,short_ans,model=gpt_qa):
  pred = model(context=context,question = question)
  short_answer = normalize_answer(short_ans)
  if evaluate([anses],[pred['answer']])['f1'] > 30 or short_answer in normalize_answer(pred['answer']): 
    return pred['score'],pred['answer']
  else:
    return -1*pred['score'],pred['answer']


In [22]:
def get_best_distractor(sample):
  context = sample.context_ori
  question = sample.question
  anses = (sample.answers)
  anses_len= [len(ans) for ans in anses]
  short_ans_i = np.argmin(anses_len)
  longest_ans_i = np.argmax(anses_len)
  short_ans,longest_ans = anses[short_ans_i], anses[longest_ans_i]
  short_distractor = get_short_distractor(question,short_ans)
  long_distractor = make_long_distractor(short_distractor)

  context_distractor_short_prefix = short_distractor +' ' + sample.context_ori
  context_distractor_short_suffix =  sample.context_ori + ' ' + short_distractor
  context_distractor_long_prefix = long_distractor +' ' + sample.context_ori
  context_distractor_long_suffix =  sample.context_ori + ' ' + long_distractor
  distractors = [short_distractor,short_distractor,long_distractor,long_distractor]
  confidence1, model_ans = correct_confidence_w_negative(sample.answers, context_distractor_short_prefix, question,short_ans)
  confidence2, model_ans = correct_confidence_w_negative(sample.answers, context_distractor_short_suffix, question,short_ans)
  confidence3, model_ans = correct_confidence_w_negative(sample.answers, context_distractor_long_prefix, question,short_ans)
  confidence4, model_ans = correct_confidence_w_negative(sample.answers, context_distractor_long_suffix, question,short_ans)
  confidences = [confidence1,confidence2,confidence3,confidence4]
  best_distractor_i = np.argmin(confidences)
  distractor = distractors[best_distractor_i]
  min_confidence = confidences[best_distractor_i]

  extended_distractor = extend_distractor(distractor)
  context_e_distractor_short_prefix = extended_distractor +' ' + sample.context_ori
  context_e_distractor_short_suffix =  sample.context_ori + ' ' + extended_distractor
  confidence5, model_ans = correct_confidence_w_negative(sample.answers, context_e_distractor_short_prefix, question,short_ans)
  confidence6, model_ans = correct_confidence_w_negative(sample.answers, context_e_distractor_short_suffix, question,short_ans)
  print(extended_distractor)
  confidences = [confidence1,confidence2,confidence3,confidence4,confidence5, confidence6]
  best_distractor_i = np.argmin(confidences)
  min_confidence = confidences[best_distractor_i]
  distractors = [short_distractor,short_distractor,long_distractor,long_distractor,extended_distractor,extended_distractor]
  best_distractor = distractors[best_distractor_i]
  return best_distractor, distractors, min_confidence,best_distractor_i,confidences
        
trigger_lexical_iterations = 3

def get_best_answer_sentence_edit(sample, context, generator):
  question = sample.question
  anses = (sample.answers)
  anses_len= [len(ans) for ans in anses]
  short_ans_i = np.argmin(anses_len)
  longest_ans_i = np.argmax(anses_len)
  short_ans,longest_ans = anses[short_ans_i], anses[longest_ans_i]
  context4s = []
  for k in range(trigger_lexical_iterations):
    context4 = add_trigger_lexical(context, question,generator=generator,longest_ans=longest_ans,temp=1.1)
    if context4:
      context4s.append(context4)
  context4s.append(context) # original context
  confidences = []
  for context4 in context4s:
    confidence4, model_ans = correct_confidence_w_negative(sample.answers, context4, question,short_ans)
    confidences.append(confidence4)
  best_context_i = np.argmin(confidences)
  best_context = context4s[best_context_i]
  print(confidences)
  return best_context,confidences[best_context_i],context4s,best_context_i

# Generate ShortcutQA

In [23]:
contexts_edited = []
distractors_lst=[]
best_distractor_i_lst=[]
i = 0

In [24]:
i

0

In [ ]:
generator = gpt4

for j in tq.tqdm(range(i,300)):
  sample = attacked_data.iloc[i]
  context = sample.context_ori
  question = sample.question
  anses = sample.answers
  anses_len= [len(ans) for ans in anses]
  short_ans_i = np.argmin(anses_len)
  longest_ans_i = np.argmax(anses_len)
  short_ans, longest_ans = anses[short_ans_i], anses[longest_ans_i]
  ori_confidence,ori_ans = correct_confidence_w_negative(sample.answers, context,question,short_ans)
  confidence = ori_confidence
  best_distractor, distractors, min_confidence,best_distractor_i,confidences1 = get_best_distractor(sample)

  if bool(best_distractor_i%2):
    context1 = sample.context_ori+' '+best_distractor
  else:
    context1 = best_distractor+' '+sample.context_ori
  
  
  confidence = min_confidence

  context3,dist_diff = add_trigger_anchor(context, longest_ans,question,trigger_full=True,generator=generator)
  

  if bool(best_distractor_i%2):
    context13 = context3+' '+best_distractor
  else:
    context13 = best_distractor+' '+context3
  
  confidence3,model_ans = correct_confidence_w_negative(sample.answers, context13, question,short_ans)
  confidence = confidence3


  context134,confidence4,context4s,best_context_i = get_best_answer_sentence_edit(sample, context13, gpt4)
  
  print(f'ori_confidence: {ori_confidence}')
  print(f'confidence: {confidence4}')
  print('')

  best_distractor_i_lst.append(best_distractor_i)
  distractors_lst.append(distractors)
  
  contexts_edited.append(context134)
  
  i+=1


In [ ]:
#Last index edited
i

In [73]:
#rename column name
attacked_data['context natural'] = attacked_data['context_ori']
attacked_data['gold_labels'] = attacked_data['answers']

In [ ]:
dataset_adv_df_subset_processed = attacked_data.iloc[0:i]
dataset_adv_df_subset_processed['best_distractor_i_lst'] = best_distractor_i_lst
dataset_adv_df_subset_processed['distractors_lst'] = distractors_lst
dataset_adv_df_subset_processed['context edited'] = contexts_edited

In [75]:
#dataset_adv_df_subset_processed.to_csv('dataset_adv_df_subset_processed_news300.csv')
